# Image upscaler

- Uses [Waifu2x](https://github.com/nagadomi/nunif) for AI image upscaling
- Based on the colab demo available on the [Real-ESRGAN](https://github.com/xinntao/Real-ESRGAN) repository

## Creating user input UI

- URLs for images go in the textbox below
- One URL per line

In [ ]:
from enum import Enum

import ipywidgets as wd

class Methods(Enum):
  NOISE_SCALE = "noise_scale"
  NOISE = "noise"
  SCALE = "scale"
  SCALE2X = "scale2x"
  SCALE4X = "scale4x"
  NOISE_SCALE2X= "noise_scale2x"
  NOISE_SCALE4X = "noise_scale4x"

class Styles(Enum):
  ART = "art"
  ART_SCAN = "art_scan"
  PHOTO = "photo"
  SCAN = "scan"
  NONE = "None"

dd_method = wd.Dropdown(
    description="Method: ",
    options = [method.value for method in Methods],
    index = 0,
)
dd_style = wd.Dropdown(
    description="Style: ",
    options = [style.value for style in Styles],
    index = 0,
)
ft_denoise = wd.IntText(
    value=0,
    description='Denoise:',
)
ta_urls = wd.Textarea(
    placeholder="Type URLs for images",
    description="URLs:",
)

display(
    ta_urls,
    dd_method,
    dd_style,
    ft_denoise,
)

## Retrieving images

If no URLs are passed to the textbox on the UI above, an upload prompt is show to get images from the user's machine

In [ ]:
from google.colab import files
import os
import shutil

links = ta_urls.value.splitlines()

upload_folder = '/content/upload'
result_folder = '/content/results'

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
os.mkdir(upload_folder)
os.mkdir(result_folder)

if len(links) == 0:
  # upload images
  uploaded = files.upload()
  for filename in uploaded.keys():
    dst_path = os.path.join(upload_folder, filename)
    print(f'move {filename} to {dst_path}')
    shutil.move(filename, dst_path)
    if filename.endswith(".zip"):
      print(f"Decompressing {filename}")
      os.system(f"unzip -j {dst_path} -d {upload_folder}")
      os.remove(dst_path)
else:
  # Download image links if set
  for link in links:
   !wget --directory-prefix {upload_folder} {link}

## Setting up the environment

In [ ]:
%%capture
!sudo apt install git-core libmagickwand-dev libraqm-dev

In [ ]:
%cd /content
!rm -rf nunif
!git clone https://github.com/nagadomi/nunif.git
%cd nunif
#!python3 -m venv .venv
#!source .venv/bin/activate

In [ ]:
%%capture
!pip3 install -r requirements-torch.txt
!pip3 install -r requirements.txt

In [ ]:
!python3 -m waifu2x.download_models
!python3 -m waifu2x.web.webgen

## Upscaling images

In [ ]:
from PIL import Image

script = 'waifu2x.cli'
image_count = len(os.listdir(upload_folder))

args = [
    f"-m {dd_method.value}",
    f"-n {ft_denoise.value}",
    (f"--style {dd_style.value}" if dd_style.value == Styles.NONE.value else ""),
]
args = " ".join(args)

!python -m {script} {args} -i {upload_folder} -o {result_folder}

## Displaying results

In [ ]:
# utils for visualization
import cv2
import matplotlib.pyplot as plt
def display_img(img1, img2):
  fig = plt.figure(figsize=(25, 10))
  ax1 = fig.add_subplot(1, 2, 1)
  plt.title('Input image', fontsize=16)
  ax1.axis('off')
  ax2 = fig.add_subplot(1, 2, 2)
  plt.title('Waifu2x output', fontsize=16)
  ax2.axis('off')
  ax1.imshow(img1)
  ax2.imshow(img2)
def imread(img_path):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

# display each image in the upload folder
import os
import glob

input_list = sorted(glob.glob(os.path.join(upload_folder, '*')))
output_list = sorted(glob.glob(os.path.join(result_folder, '*')))
for input_path, output_path in zip(input_list, output_list):
  img_input = imread(input_path)
  img_output = imread(output_path)
  display_img(img_input, img_output)

## Downloading results

In [ ]:
# Download the results
zip_filename = 'Waifu2x_result.zip'
if os.path.exists(zip_filename):
  os.remove(zip_filename)
os.system(f"zip -r -j {zip_filename} {result_folder}/*")
files.download(zip_filename)